## Import necessary modules and data

In [11]:
import numpy as np
import pandas as pd
import sklearn
import keras
import seaborn
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
#!pip install -q keras_metrics
import keras_metrics
from sklearn.model_selection import cross_validate
#!pip install livelossplot
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import precision_recall_fscore_support as score
import random


In [12]:
url = 'https://raw.githubusercontent.com/Dalthirr/NeuralNetworks/master/Churn_Modelling.csv'
#Pobieranie danych
df = pd.read_csv(url)
df = df.drop(["CustomerId", "Surname", "RowNumber"],
             axis=1)
df = pd.concat([df, pd.get_dummies(df[["Gender", "Geography"]], drop_first=True)],
               sort=False, axis=1).drop(df[["Gender", "Geography"]], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          10000 non-null int64
Age                  10000 non-null int64
Tenure               10000 non-null int64
Balance              10000 non-null float64
NumOfProducts        10000 non-null int64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Gender_Male          10000 non-null uint8
Geography_Germany    10000 non-null uint8
Geography_Spain      10000 non-null uint8
dtypes: float64(2), int64(7), uint8(3)
memory usage: 732.5 KB


## Tworzymy zbiór treningowy i testowy, standaryzujemy i stratyfikujemy odpowiednio do proporcji etykiet

In [13]:
X = df.drop(['Exited'], axis=1)
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

scaler = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)

X_test_std = scaler.transform(X_test)

X_std = scaler.transform(X)


/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  import sys
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


Zbuduję przykładową sieć, która składa się z warstwy wejściowej, jednej warstwy ukrytej i  warstwy wyjściowej.
Kompilujemy model przy użyciu 

loss_function='binary_crossentropy';

optimizer='Adam'.


Naszą główną metryką będzie **precyzja** oznaczająca procent osób prawidłowie określonych jako zagrożone odejściem.

In [14]:
# Pierwsze podejście do modelu
# Creating Model, I will build simple networt using Sequential.
model = Sequential()
# Add an input layer shape depends on shape of X. We have 11 columns in X.
model.add(Dense(11, activation='relu', input_shape=(11,)))

# Add one hidden layer 
model.add(Dense(121, activation='relu'))



# Add an output layer 
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(),'accuracy'])
                   
model.fit(X_train_std, y_train,epochs=3, batch_size=1, verbose=1, validation_data=(X_test,y_test))

model.evaluate(X_test_std,y_test)

UnboundLocalError: local variable 'arrays' referenced before assignment

In [15]:
#Sprawdźmy skuteczność naszej predykcji - raportem klasyfikacyjnym
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test,y_pred)
report1 = classification_report(y_test,y_pred,output_dict=True)
print(report)
print(report1['0']['precision'])


3300/3300 [==============================] - 9s 3ms/step
              precision    recall  f1-score   support

           0       0.77      0.51      0.62      2628
           1       0.17      0.39      0.23       672

   micro avg       0.49      0.49      0.49      3300
   macro avg       0.47      0.45      0.43      3300
weighted avg       0.64      0.49      0.54      3300

0.7660056657223796


In [16]:
# Sprawdźmy, jak prezentują się wyniki w zależności od liczby
# neuronów w warstwie ukrytej.
nodes = [10, 30, 50, 70, 90, 110]
wyniki = {}
for i in nodes:
    model = 0
    # Creating Model, I will build simple networt using Sequential.
    model = Sequential()
    # Add an input layer shape depends on shape of X. We have 11 columns in X.
    model.add(Dense(11, activation='relu', input_shape=(11,)))

    # Add one hidden layer
    model.add(Dense(i, activation='relu'))

    # Add an output layer
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[keras_metrics.precision(), 'accuracy'])

    model.fit(X_train_std, y_train, epochs=5, batch_size=1, verbose=1, validation_data=(X_test, y_test))
    print("Zakończono trenowanie modelu z {0} węzłami.".format(i))
    model.evaluate(X_test_std, y_test)
    y_pred = model.predict_classes(X_test_std)
    report = classification_report(y_test, y_pred, output_dict=True)
    wyniki.update({i: [report['0']['precision'], report['1']['precision']]})

print(wyniki)


UnboundLocalError: local variable 'arrays' referenced before assignment

In [17]:
# Wybieramy model z 90 węzłami - dał najlepsze wyniki
from keras.callbacks import EarlyStopping

model = Sequential()
# Add an input layer shape depends on shape of X. We have 11 columns in X.
model.add(Dense(11, activation='relu', input_shape=(11,)))
model.add(Dropout(0.1, ))
# Add one hidden layer 
model.add(Dense(90, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(), 'accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train_std, y_train, epochs=30, batch_size=1, verbose=1, validation_data=(X_test, y_test),
          callbacks=[early_stopping_monitor])

model.evaluate(X_test_std, y_test)
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test, y_pred)
report1 = classification_report(y_test, y_pred, output_dict=True)
print(report)



UnboundLocalError: local variable 'arrays' referenced before assignment

### autor: Tomasz Sołtysiak